In [ ]:
#describe the data
def descriptive_stats(df): 
    print('This is the descriptive statistics function')
    print(df.columns) 

    # Create a pivot table to tabulate the data by year and month
    pivot_table = df.pivot_table(index='Year', columns='Month', aggfunc='size', fill_value=0)

    # Display the pivot table
    print(pivot_table)

    df['ProcedenciaOC'].unique()
    df['precioNeto'] = pd.to_numeric(df['precioNeto'], errors='coerce')
    df = df.dropna(subset=['precioNeto'])
    total_weight = df['precioNeto'].sum()

    convenio_marco_weight = df[df['ProcedenciaOC'] == 'Proveniente de convenio marco']['precioNeto'].sum()
    share_convenio_marco = convenio_marco_weight / total_weight
    print('share of total expenditure in FAs:', share_convenio_marco)

#drop purchases that do not come from FAs or that are not from vehicles. 
def keep_FA_vehicles(df): 
    #remove purchases that are not trough FA
    df = df[df['ProcedenciaOC'] == 'Proveniente de convenio marco']


    ## see which variables are give you information of the type of product being bougth 
    varlist = [ 'Categoria',  'NombreroductoGenerico', 'RubroN1', 'RubroN2', 'RubroN3']
    #for var in varlist:
        #print('The var is: ', var)
        #print(df[var].unique())


    # create a df with the rows that could car purchases 
    columns_to_check = ['Categoria', 'NombreroductoGenerico', 'RubroN1', 'RubroN2', 'RubroN3']
    mask = df[columns_to_check].apply(lambda x: x.str.contains('Vehículos|vehículos', case=False, na=False)).any(axis=1)
    df = df[mask]

    return df

# summarize the amounts of the purchases and transforms amount variables to int, input: dictionary with dataframes
def summarize_amounts(df_dict): 
    for year, df in df_dict.items():
        print(f"Year: {year}")
        
        #get sense of price and its dispersion 
        df['MontoTotalOC'] = pd.to_numeric(df['MontoTotalOC'], errors='coerce')
        summary = df['MontoTotalOC'].describe()
        print(summary)

        df['precioNeto'] = pd.to_numeric(df['precioNeto'], errors='coerce')
        summary = df['precioNeto'].describe()
        print(summary)

        df_dict[year] = df
    return df_dict


# Find the values of 'Codigo' that are repeated more than N times
def repeated_codes(df_dict, N = 3): 
    for year, df in df_dict.items():
        print(f"Year: {year}")
        repeated_codigos = df['Codigo'].value_counts()
        repeated_codigos = repeated_codigos[repeated_codigos > N]
        print(repeated_codigos)
       
    return 


In [2]:
import pandas as pd
from pandasgui import show
import os

type = 'CM'
# create the root path 
current_dir = os.path.dirname(os.getcwd())
base_path = os.path.join(current_dir,'raw_data', 'ConvenioMarco', 'transacciones')

#joint the files for a single year. 
def join_data(base_path, year): 
    # Create an empty list to store DataFrames
    dfs = []

    for year in range(year, year+1 ):
        for month in range(1, 13):
            # Construct the file path
            file_path = os.path.join(base_path, str(year), f"{year}-{month}.csv")
            print(f"Reading file: {file_path}")
            # Check if the file exists before trying to read it
            if os.path.exists(file_path):
                try:
                    df = pd.read_csv(file_path, encoding='utf-8')
                    
                    # Add year and month columns for reference
                    df['Year'] = year
                    df['Month'] = month
                    
                    # Append the DataFrame to the list
                    dfs.append(df)
                    print(f"Successfully read file: {file_path}")
                except Exception as e:
                    print(f"Error reading file {file_path}: {str(e)}")
            else:
                print(f"File not found: {file_path}")

    # Concatenate all DataFrames in the list
    combined_df = pd.concat(dfs, ignore_index=True)

    # Display basic information about the combined DataFrame
    print(combined_df.info())
    return combined_df

# show the columns that have relevant information 
def show_df(df): 

    # List of columns we care about 
    columns_to_check = ['ID', 'Codigo', 'Link', 'Nombre', 'Descripcion/Obervaciones', 'MontoTotalOC', 'MontoTotalOC_PesosChilenos', 'Impuestos','TotalNetoOC', 'CodigoProveedor',   'RegionProveedor',  'IDItem', 'codigoCategoria', 'Categoria', 'codigoProductoONU', 'NombreroductoGenerico', 'RubroN1', 'RubroN2', 'RubroN3', 'EspecificacionComprador', 'EspecificacionProveedor', 'cantidad',  'precioNeto', 'totalCargos','totalDescuentos', 'totalImpuestos', 'totalLineaNeto']

    df_show = df[columns_to_check]
    show(df_show)

#save the dataframe, otherwise one would have to use the join_data function each time
def save_df(df_light, year, type):    
    try:
        os.makedirs('interm_data', exist_ok=True)
        save_path = os.path.join('interm_data', 'search_CM', f'{type}_{year}.csv')
        
        # If successful, now try to save your DataFrame
        df_light.to_csv(save_path, index=False)
        print(f"Successfully saved DataFrame to: {save_path}")

    except PermissionError as e:
        print(f"Permission error: {e}")
        print("You don't have permission to write to this location.")
    except FileNotFoundError as e:
        print(f"File not found error: {e}")
        print("The specified path doesn't exist and couldn't be created.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

 

years = range(2020, 2021)
df_dict = {}  # Initialize an empty dictionary



for year in years:
    exists = False 
    combined_file_path = os.path.join(current_dir,'interm_data', 'search_CM', f'{type}_{year}.csv')
    print(f"Absolute path of combined file for {year}: {os.path.abspath(combined_file_path)}")
    
    #check if the file already exists
    if os.path.exists(combined_file_path):
        print(f"Combined file for {year} already exists: {combined_file_path}") # If the file exists, print a message
        exists = True 
        df_light = pd.read_csv(combined_file_path) # If the file exists, read it into a DataFrame
       
    #if the file does not exist, create it
    if exists == False: 

        df = join_data(base_path, year)

         

Absolute path of combined file for 2020: c:\Users\lucas\OneDrive - Yale University\Documents\GitHub\2nd-year-paper\interm_data\search_CM\CM_2020.csv
Reading file: c:\Users\lucas\OneDrive - Yale University\Documents\GitHub\2nd-year-paper\raw_data\ConvenioMarco\transacciones\2020\2020-1.csv
Successfully read file: c:\Users\lucas\OneDrive - Yale University\Documents\GitHub\2nd-year-paper\raw_data\ConvenioMarco\transacciones\2020\2020-1.csv
Reading file: c:\Users\lucas\OneDrive - Yale University\Documents\GitHub\2nd-year-paper\raw_data\ConvenioMarco\transacciones\2020\2020-2.csv
Successfully read file: c:\Users\lucas\OneDrive - Yale University\Documents\GitHub\2nd-year-paper\raw_data\ConvenioMarco\transacciones\2020\2020-2.csv
Reading file: c:\Users\lucas\OneDrive - Yale University\Documents\GitHub\2nd-year-paper\raw_data\ConvenioMarco\transacciones\2020\2020-3.csv
Successfully read file: c:\Users\lucas\OneDrive - Yale University\Documents\GitHub\2nd-year-paper\raw_data\ConvenioMarco\trans

C:\Users\lucas\AppData\Local\Temp\ipykernel_40520\3122179320.py:23: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='utf-8')


Successfully read file: c:\Users\lucas\OneDrive - Yale University\Documents\GitHub\2nd-year-paper\raw_data\ConvenioMarco\transacciones\2020\2020-6.csv
Reading file: c:\Users\lucas\OneDrive - Yale University\Documents\GitHub\2nd-year-paper\raw_data\ConvenioMarco\transacciones\2020\2020-7.csv
Successfully read file: c:\Users\lucas\OneDrive - Yale University\Documents\GitHub\2nd-year-paper\raw_data\ConvenioMarco\transacciones\2020\2020-7.csv
Reading file: c:\Users\lucas\OneDrive - Yale University\Documents\GitHub\2nd-year-paper\raw_data\ConvenioMarco\transacciones\2020\2020-8.csv
Successfully read file: c:\Users\lucas\OneDrive - Yale University\Documents\GitHub\2nd-year-paper\raw_data\ConvenioMarco\transacciones\2020\2020-8.csv
Reading file: c:\Users\lucas\OneDrive - Yale University\Documents\GitHub\2nd-year-paper\raw_data\ConvenioMarco\transacciones\2020\2020-9.csv
Successfully read file: c:\Users\lucas\OneDrive - Yale University\Documents\GitHub\2nd-year-paper\raw_data\ConvenioMarco\tra

C:\Users\lucas\AppData\Local\Temp\ipykernel_40520\3122179320.py:23: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='utf-8')


Successfully read file: c:\Users\lucas\OneDrive - Yale University\Documents\GitHub\2nd-year-paper\raw_data\ConvenioMarco\transacciones\2020\2020-10.csv
Reading file: c:\Users\lucas\OneDrive - Yale University\Documents\GitHub\2nd-year-paper\raw_data\ConvenioMarco\transacciones\2020\2020-11.csv


C:\Users\lucas\AppData\Local\Temp\ipykernel_40520\3122179320.py:23: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding='utf-8')


Successfully read file: c:\Users\lucas\OneDrive - Yale University\Documents\GitHub\2nd-year-paper\raw_data\ConvenioMarco\transacciones\2020\2020-11.csv
Reading file: c:\Users\lucas\OneDrive - Yale University\Documents\GitHub\2nd-year-paper\raw_data\ConvenioMarco\transacciones\2020\2020-12.csv
Successfully read file: c:\Users\lucas\OneDrive - Yale University\Documents\GitHub\2nd-year-paper\raw_data\ConvenioMarco\transacciones\2020\2020-12.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1902031 entries, 0 to 1902030
Data columns (total 48 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   Unnamed: 0                    int64  
 1   Nro Licitación Pública        object 
 2   Id Convenio Marco             int64  
 3   Convenio Marco                object 
 4   CodigoOC                      object 
 5   NombreOC                      object 
 6   Fecha Envío OC                object 
 7   EstadoOC                      object 
 8   Provie

In [3]:
df

,Unnamed: 0,Nro Licitación Pública,Id Convenio Marco,Convenio Marco,CodigoOC,NombreOC,Fecha Envío OC,EstadoOC,Proviene de Gran Compra,idGranCompra,...,Comuna del Proveedor,Región del Proveedor,Observaciones,Forma de Pago,Orgcode_Comprador,Entcode_Comprador,Orgcode_Proveedor,Entcode_Proveedor,Year,Month
0,0,2239-4-LP14,5800181,CM Articulos de Escritorio y Papeleria,1545-9099-CM18,2239-4-LP14 Articulos de escritorio y papeleria,2020-01-06,Aceptada,No,NaN,...,Santiago,Región Metropolitana de Santiago,ADQ. PLASTIFICADORA PARA UNIDAD DE ALIMENTACIO...,30 días contra la recepción conforme de la fac...,2541.0,7393.0,292211,286118,2020,1
1,1,2239-16-LR15,5800221,"Órtesis, Prótesis, Endoprótesis e Insumos de S...",2157-684-CM19,"2239-16-LR15 Órtesis, Prótesis, Endoprótesis e...",2020-01-27,Aceptada,No,NaN,...,Santiago,Región Metropolitana de Santiago,COT.35166 IC.10112 PCTE. J.M.V,NaN,3152.0,7479.0,270842,265250,2020,1
2,2,2239-16-LR15,5800221,"Órtesis, Prótesis, Endoprótesis e Insumos de S...",2157-692-CM19,"2239-16-LR15 Órtesis, Prótesis, Endoprótesis e...",2020-01-27,Recepción Conforme,No,NaN,...,Santiago,Región Metropolitana de Santiago,COT.35165 IC.9861 PCTE. S.M.M,NaN,3152.0,7479.0,270842,265250,2020,1
3,3,2239-14-LP13,5800176,"Servicio de Courier, Operador Logistico y Serv...",1067455-78-CM19,"2239-14-LP13 Servicio Courier, Operador Logist...",2020-01-09,Aceptada,No,NaN,...,Santiago,Región Metropolitana de Santiago,Orden de Compra generada desde carro compras,30 días contra la recepción conforme de la fac...,1067455.0,1659065.0,15294,16120,2020,1
4,4,2239-5-LP14,5800178,"Convenio Marco de Venta, Arriendo y Suministro...",3684-141-CM19,MATERIAL DE IMPRESION PARA LAS DEPENDENCIAS DE...,2020-01-02,Aceptada,No,NaN,...,Conchalí,Región Metropolitana de Santiago,Adquisición de material de impresión para las ...,30 días contra la recepción conforme de la fac...,4604.0,116298.0,70073,67999,2020,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1902026,188748,2239-23-LP10,5800143,PASAJES AEREOS NACIONALES REGULARES,677-229-CM20,Orden de Compra,2020-12-31,Recepción Conforme,No,NaN,...,Santiago,13,OC por CM:203 Reserva:FRCQMS,Contra Factura 60 Días,1673.0,6941.0,25402,25638,2020,12
1902027,188749,2239-23-LP10,5800143,PASAJES AEREOS NACIONALES REGULARES,677-229-CM20,Orden de Compra,2020-12-31,Recepción Conforme,No,NaN,...,Santiago,13,OC por CM:203 Reserva:FRCQMS,Contra Factura 60 Días,1673.0,6941.0,25402,25638,2020,12
1902028,188750,2239-17-LP13,5800198,"Lentes, Audífonos e Implantes Auditivos (AUDIF...",3921-1364-CM20,"Resolutividad, componente otorrino, audífonos",2020-12-31,Aceptada,No,NaN,...,Providencia,Región Metropolitana de Santiago,Se requiere para dar cumplimiento a componente...,30 días contra la recepción conforme de la fac...,4841.0,118084.0,30752,30632,2020,12
1902029,188751,2239-17-LP13,5800198,"Lentes, Audífonos e Implantes Auditivos (AUDIF...",1057509-8545-CM20,ADQ. AUDIFONOS ENERO2021(CM-NVS),2020-12-31,Aceptada,No,NaN,...,Providencia,Región Metropolitana de Santiago,CONVENIO MARCO\r\nOC ANITA 22087785\r\nAUDIFON...,30 días contra la recepción conforme de la fac...,1057509.0,7042.0,632295,1235540,2020,12


In [8]:
## see which variables give you information of the type of product being bougth 
varlist = [ 'Categoria',  'NombreroductoGenerico', 'RubroN1', 'RubroN2', 'RubroN3']
    
for year, df in df_dict.items():
    print(f"Year: {year}")
    
    for var in varlist:
        print('The var is: ', var)
        print(df_light[var].unique())

    df = df[(df['RubroN2'] == 'Vehículos motorizados')& (df['NombreroductoGenerico'] != 'Camiones de transporte')]
    print(df.info())

    df_dict[year] = df

Year: 2017
The var is:  Categoria
['Vehículos y equipamiento en general / Vehículos motorizados / Vehículos para pasajeros'
 'Vehículos y equipamiento en general / Vehículos motorizados / Vehículos de emergencia'
 'Vehículos y equipamiento en general / Vehículos motorizados / Vehículos para turismo'
 'Vehículos y equipamiento en general / Vehículos motorizados / Vehículos especiales y de recreación'
 'Vehículos y equipamiento en general / Vehículos motorizados / Bicicletas con motor']
The var is:  NombreroductoGenerico
['SUV o todo terrenos'
 'Camiones ligeros o vehículos de deporte de utilida' 'Ambulancias'
 'Automóviles' 'Trineos motorizados o motos de nieve'
 'Escuters o motos pequeñas']
The var is:  RubroN1
['Vehículos y equipamiento en general']
The var is:  RubroN2
['Vehículos motorizados']
The var is:  RubroN3
['Vehículos para pasajeros' 'Vehículos de emergencia'
 'Vehículos para turismo' 'Vehículos especiales y de recreación'
 'Bicicletas con motor']
<class 'pandas.core.frame.D

In [9]:
df_dict = summarize_amounts(df_dict)

Year: 2017
count    2.270000e+02
mean     1.085645e+08
std      1.711052e+08
min      7.000000e+03
25%      3.302190e+06
50%      3.705017e+07
75%      1.344164e+08
max      1.057077e+09
Name: MontoTotalOC, dtype: float64
count    1.114000e+03
mean     1.420325e+07
std      1.138950e+07
min      2.479000e+03
25%      8.649521e+06
50%      1.419249e+07
75%      1.789000e+07
max      1.182000e+08
Name: precioNeto, dtype: float64
Year: 2018
count    5.560000e+02
mean     4.325683e+07
std      1.129131e+08
min      4.498200e+04
25%      1.549772e+07
50%      2.027397e+07
75%      3.047704e+07
max      1.336477e+09
Name: MontoTotalOC, dtype: float64
count    9.810000e+02
mean     1.410379e+07
std      9.113468e+06
min      5.308000e+03
25%      9.866050e+06
50%      1.444000e+07
75%      1.738500e+07
max      1.265966e+08
Name: precioNeto, dtype: float64
Year: 2019
count    9.210000e+02
mean     4.808638e+07
std      1.339362e+08
min      5.057500e+04
25%      1.529460e+07
50%      2.000000

In [10]:
repeated_codes(df_dict, N = 3)

Year: 2017
Codigo
918434-343-CM17    15
701-1622-CM17      12
612-14-CM17        11
904268-119-SE17     9
587-1843-CM17       7
1596-275-CM17       4
612-2817-SE17       4
1591-380-CM17       4
Name: count, dtype: int64
Year: 2018
Codigo
612-72-CM18       27
587-2780-CM18      6
4629-1271-CM18     6
5186-285-CM18      5
731-1559-CM18      4
5693-267-SE18      4
708-1011-CM18      4
Name: count, dtype: int64
Year: 2019
Codigo
612-153-SE19       26
4629-1204-CM19     12
585-211-CM19        7
587-1745-CM19       6
1080094-7-SE19      5
587-2528-CM19       5
593951-549-CM19     4
3686-3-CM19         4
1080094-61-SE19     4
1080095-10-SE19     4
587-2950-CM19       4
3422-109-SE19       4
731-1265-CM19       4
2084-453-SE19       4
1591-319-CM19       4
Name: count, dtype: int64
Year: 2020
Codigo
593951-231-CM20     8
1098710-128-CM20    5
5418-419-SE20       4
2511-122-SE20       4
Name: count, dtype: int64
Year: 2021
Series([], Name: count, dtype: int64)
Year: 2022
Series([], Name: count,

In [12]:
show_df(df_dict[2023])

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
